In [1]:
import numpy as np
import pandas as pd
import sys
import seaborn as sns
import os.path
import time, itertools, re
from collections import Counter
import scipy as sc
from sklearn.metrics import make_scorer, balanced_accuracy_score, confusion_matrix, root_mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold
from sklearn.pipeline import make_pipeline, Pipeline
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
print(os.getcwd())
fileloc_data='/'.join(os.getcwd().split('/')[0:5])+ '/data/annonymizedDatasets/'
savetag='pred_lavSQ_MHC'
print(fileloc_data)
code_path='/'.join(os.getcwd().split('/')[0:4])+'/sklvq/'
sys.path.append(code_path)
from sklvq import GMLVQ, LGMLVQ
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from HypOpt import GridSearchClassifiers, GridSearch_LVQ


/home/sreejita/ProjectsPy/GGZ/code_scripts/ED-profiling
/home/sreejita/ProjectsPy/GGZ/data/annonymizedDatasets/


In [2]:
colsLAV=['Lav-Neg_Waardering', 'Lav-Gebrek_Vertrouwdheid', 'Lav-Alg_Ontevredenheid']
colsSQ48=['SQ48-Vijandigheid','SQ48-Agorafobie','SQ48-Angst','SQ48-Depressie', 'SQ48-Cognitieve_Klachten',
          'SQ48-Somatische_Klachten', 'SQ48-Sociale_Fobie','SQ48-Vitaliteit_Optimisme', 'SQ48-Werk_Studie']
colsMHCSF=['MHCSF-EmotionWB', 'MHCSF-SocialWB', 'MHCSF-PsychWB']
cols2consider=['Main-Age','Main-ED_Codes', 'EDEQ-Score']+colsLAV+colsSQ48+colsMHCSF
cols2train=['Main-Age']+colsLAV+colsSQ48+colsMHCSF
colNames=['Main-Age', 'Main-ED_Codes', 'EDEQ-Score','Lav-Neg_Waardering','Lav-Gebrek_Vertrouwdheid', 'Lav-Alg_Ontevredenheid', 
'SQ48-Hostility','SQ48-Agorafobie','SQ48-Angst','SQ48-Depressie','SQ48-Cognitieve_Complain',
'SQ48-Somatische_Complain', 'SQ48-Sociale_Fobie','SQ48-Vitaliteit_Optimisme','SQ48-Werk_Studie', 
'MHCSF-EmotionWB', 'MHCSF-SocialWB', 'MHCSF-PsychWB'] 
colRename_dict=dict(zip(cols2consider, colNames))
df_adapted_combo=pd.read_csv(fileloc_data+'maskedDAIsy_MainED_Lav_SQ48_MHCSF_Visit1.tsv', sep='\t', decimal=',')
df_adapted_combo['EDtype'] = df_adapted_combo['EDtype'].astype('category')
df_adapted_combo.astype(dict(zip(colsLAV, ['float', 'float','float']))).dtypes
df_adapted_combo.astype(dict(zip(colsSQ48,['float', 'float','float','float', 'float','float','float', 'float','float']))).dtypes
df_adapted_combo.astype(dict(zip(colsMHCSF, ['float', 'float','float']))).dtypes
df_adapted_combo.rename(columns=colRename_dict, inplace=True)
adapted_combo_cols=['Main-Age','Lav-Alg_Ontevredenheid', 'Lav-Gebrek_Vertrouwdheid','Lav-Neg_Waardering',
    'SQ48-Hostility','SQ48-Agorafobie','SQ48-Angst','SQ48-Depressie','SQ48-Cognitieve_Complain',
    'SQ48-Somatische_Complain', 'SQ48-Sociale_Fobie','SQ48-Vitaliteit_Optimisme','SQ48-Werk_Studie', 'MHCSF-EmotionWB', 
    'MHCSF-SocialWB', 'MHCSF-PsychWB']
#df_adapted_combo['ED_Codes'] = df_adapted_combo['EDtype'].cat.codes

# Classification

In [3]:
df_train_adapted=df_adapted_combo[adapted_combo_cols].loc[df_adapted_combo['Split']=='Train']
YTrain=df_adapted_combo['EDtype'].loc[df_adapted_combo['Split']=='Train']#.to_numpy()
df_test_adapted=df_adapted_combo[adapted_combo_cols].loc[df_adapted_combo['Split']=='Test']
YTest=df_adapted_combo['EDtype'].loc[df_adapted_combo['Split']=='Test']#.to_numpy()

In [4]:
sampling_strategy='not majority'
GridSearchClassifiers(df_train_adapted, YTrain,sampling_strategy)

KeyboardInterrupt: 

In [ ]:
GridSearch_LVQ(df_train_adapted, YTrain,sampling_strategy)

In [6]:

df_train_eds=df_adapted_combo[adapted_combo_cols].loc[(df_adapted_combo['Split']=='Train') & (df_adapted_combo['EDtype']!='Others')]
YTrain_eds=df_adapted_combo['EDtype'].loc[(df_adapted_combo['Split']=='Train') & (df_adapted_combo['EDtype']!='Others')]#.to_numpy()
df_test_eds=df_adapted_combo[adapted_combo_cols].loc[(df_adapted_combo['Split']=='Test') & (df_adapted_combo['EDtype']!='Others')]
YTest_eds=df_adapted_combo['EDtype'].loc[(df_adapted_combo['Split']=='Test') & (df_adapted_combo['EDtype']!='Others')]#.to_numpy()
GridSearchClassifiers(df_train_eds, YTrain_eds, sampling_strategy)
GridSearch_LVQ(df_train_eds, YTrain_eds, sampling_strategy)

RF: 
   n_Trees Max_Features       MAA
9     250           11  0.708000
7     500            7  0.702667
1     250            5  0.702667
6     300            7  0.700667
KNN: 
    K Dist Metric       MAA
0  3   minkowski  0.628667
3  3      cosine  0.599333
1  5   minkowski  0.560667
4  5      cosine  0.536000
LDA: 
   Solver       MAA
0    svd  0.489333
1   lsqr  0.489333
2  eigen  0.489333
Linear SVM: 
      C       MAA
5  100  0.516000
4   10  0.512000
3    1  0.508000
2  0.1  0.496667
RBF SVM: 
       C Gamma       MAA
27   10     1  0.728000
33  100     1  0.728000
21    1     1  0.702667
32  100  0.01  0.690000
GMLVQ: 
     Prot per cls Solver Num Component Activation type       MAA
42  [2, 2, 2, 2]  lbfgs            11           swish  0.586000
46  [2, 2, 2, 2]  lbfgs            15           swish  0.572000
38  [2, 2, 2, 2]  lbfgs             7           swish  0.570000
43  [2, 2, 2, 2]   adam            11           swish  0.565333


# Regression

In [4]:
from HypOptRegress import GridSearchRegressors
df_train_adapted=df_adapted_combo[adapted_combo_cols].loc[df_adapted_combo['Split']=='Train']
YTrain_reg=df_adapted_combo['EDEQ-Score'].loc[df_adapted_combo['Split']=='Train']#.to_numpy()
df_test_adapted=df_adapted_combo[adapted_combo_cols].loc[df_adapted_combo['Split']=='Test']
YTest_reg=df_adapted_combo['EDEQ-Score'].loc[df_adapted_combo['Split']=='Test']
GridSearchRegressors(df_train_adapted, YTrain_reg.to_numpy())

KNN: 
    K Dist Metric     Score  Abs_Score
1  5   minkowski -0.228309   0.228309
2  7   minkowski -0.231111   0.231111
0  3   minkowski -0.236836   0.236836
4  5      cosine -0.237039   0.237039
RF: 
     n_Trees  Max_Features       Criterion     Score  Abs_Score
2       300             5   squared_error -0.221379   0.221379
18      300             5  absolute_error -0.223021   0.223021
3       500             5   squared_error -0.223269   0.223269
53      250             7         poisson -0.223446   0.223446
Linear SVM: 
        C     Score  Abs_Score
5  100.0 -0.227634   0.227634
3    1.0 -0.227678   0.227678
4   10.0 -0.227782   0.227782
2    0.1 -0.228291   0.228291
RBF SVM: 
         C   Gamma     Score  Abs_Score
26   10.0  0.0100 -0.217149   0.217149
31  100.0  0.0010 -0.218077   0.218077
30  100.0  0.0001 -0.232769   0.232769
25   10.0  0.0010 -0.233224   0.233224


In [5]:
df_train_eds=df_adapted_combo[adapted_combo_cols].loc[(df_adapted_combo['Split']=='Train') & (df_adapted_combo['EDtype']!='Others')]
YTrain_eds=df_adapted_combo['EDEQ-Score'].loc[(df_adapted_combo['Split']=='Train') & (df_adapted_combo['EDtype']!='Others')]#.to_numpy()
df_test_eds=df_adapted_combo[adapted_combo_cols].loc[(df_adapted_combo['Split']=='Test') & (df_adapted_combo['EDtype']!='Others')]
YTest_eds=df_adapted_combo['EDEQ-Score'].loc[(df_adapted_combo['Split']=='Test') & (df_adapted_combo['EDtype']!='Others')]#.to_numpy()

GridSearchRegressors(df_train_eds, YTrain_eds.to_numpy())

KNN: 
    K Dist Metric     Score  Abs_Score
1  5   minkowski -0.236168   0.236168
2  7   minkowski -0.242994   0.242994
4  5      cosine -0.253332   0.253332
5  7      cosine -0.254870   0.254870
RF: 
     n_Trees  Max_Features      Criterion     Score  Abs_Score
32      100             5   friedman_mse -0.229198   0.229198
42      300            11   friedman_mse -0.229444   0.229444
55      500             7        poisson -0.230135   0.230135
5       250             7  squared_error -0.230182   0.230182
Linear SVM: 
        C     Score  Abs_Score
2    0.1 -0.242192   0.242192
3    1.0 -0.242339   0.242339
4   10.0 -0.242790   0.242790
5  100.0 -0.242909   0.242909
RBF SVM: 
         C   Gamma     Score  Abs_Score
26   10.0  0.0100 -0.226652   0.226652
31  100.0  0.0010 -0.229631   0.229631
30  100.0  0.0001 -0.244207   0.244207
25   10.0  0.0010 -0.244355   0.244355
